In [1]:
# Decoding Attributes
# setting up dictionaries for each attribute
hairs = {
    0: 'wavy',
    1: 'curly',
    2: 'beanie',
    3: 'cap'
}
colors = {
    0: 'red',
    1: 'blue',
    2: 'yellow',
    3: 'green'
}
accessories = {
    0: 'sneakers',
    1: 'bowtie',
    2: 'sunglasses',
    3: 'scarf'
}
socials = {
    0: 'tiktok',
    1: 'instagram',
    2: 'discord',
    3: 'snapchat'
}


# function to decode the attributes
def zbini_attrs(type_id):
    # checking if the type_id is valid
    if type_id < 0 or type_id > 255:
        return None
    # decoding the attributes
    # hair_id is the result of type_id // 64
    hair = hairs[type_id // 64]
    # color_id is the result of (type_id % 64) // 16
    color = colors[(type_id % 64) // 16]
    # accessory_id is the result of (type_id % 16) // 4
    accessory = accessories[(type_id % 16) // 4]
    # social_id is the result of type_id % 4
    social = socials[type_id % 4]
    return hair, color, accessory, social

print(zbini_attrs(0))
print(zbini_attrs(108))
print(zbini_attrs(148))
print(zbini_attrs(248))
# print(zbini_attrs(96))
# print(zbini_attrs(351))

('wavy', 'red', 'sneakers', 'tiktok')
('curly', 'yellow', 'scarf', 'tiktok')
('beanie', 'blue', 'bowtie', 'tiktok')
('cap', 'green', 'sunglasses', 'tiktok')


In [2]:
# Checking Group Validity
def valid_study_group(zbinis, group):
    # checking if the length of group is valid
    if len(group) < 3 or len(group) > 4:
        return False, None
    # get all zbini form the group and zbinis
    zbini = [zbinis[i] for i in group]
    # get the index from zbini
    index_members = [(i[0]) for i in zbini]
    # get the attributes from the index members
    attrs = [zbini_attrs(i) for i in index_members]
    # checking if the attributes are unique
    if len(attrs) != len(set(attrs)):
        return False, None
    # put the first set of attributes in the intersection
    intersection = set(zbini[0][1])
    # do for loop in the rest of the zbini
    for i in zbini[1:]:
        # obtain the intersection of the attributes
        intersection = intersection.intersection(set(i[1]))
    # checking if the intersection is empty
    if len(intersection) == 0:
        return False, None
    return True, len(intersection)
    
example_zbinis = [(0, ['FoC', 'Calc 1', 'Logic']), (108, ['FoA', 'Calc 2', 'Logic']), (148, ['FoC', 'Calc 1', 'Logic']), (248, ['FoC', 'Calc 1', 'Logic']), (0, ['Calc 2', 'History', 'Politics']), (108, ['FoC', 'Calc 1', 'Logic'])]
print(valid_study_group(example_zbinis, (0, 1, 3)))
# (True, 1)
print(valid_study_group(example_zbinis, (0, 1, 5)))
# (False, None)
print(valid_study_group(example_zbinis, (0, 1, 2, 3)))
# (True, 1)
print(valid_study_group(example_zbinis, (1, 2, 3, 4)))
# (False, None)

(True, 1)
(False, None)
(True, 1)
(False, None)


In [9]:
# Possible Study Groups
from itertools import combinations


def possible_study_groups(zbinis):
    # create a list for storing valid groups
    groups = []
    # generate indexes from the length of zbinis
    possible_index = [i for i in range(len(zbinis))]
    # generate all possible groups of three members
    possible_groups_three = list(combinations(possible_index, 3))
    # generate all possible groups of four members
    possible_groups_four = list(combinations(possible_index, 4))
    # combine the possible groups
    possible_groups = possible_groups_four + possible_groups_three
    for i in possible_groups:
        # check if the group is valid using the valid_study_group function
        tf, num = valid_study_group(zbinis, i)
        if tf:
            # calculate the score
            score = num * 3 + 1 if len(i) == 3 else num * 3 + 2
            # append the group and score to the groups list
            groups.append((i, score))
    return groups

print(possible_study_groups([(198, ['FoC']), (138, ['FoC', 'Calc 1']), (14, ['FoC', 'Calc 1']), (66, ['Calc 1'])]))
# [((0, 1, 2), 4), ((1, 2, 3), 4)]
print(possible_study_groups([(198, ['FoC']), (138, ['FoC', 'Calc 1']), (14, ['FoC']), (66, ['FoC'])]))
# [((0, 1, 2, 3), 5), ((0, 1, 2), 4), ((0, 1, 3), 4), ((0, 2, 3), 4), ((1, 2, 3), 4)]
print(possible_study_groups([(198, ['FoC']), (138, ['Calc 1']), (14, ['Calc 1']), (66, ['FoC'])]))
# []

[((0, 1, 2), 4), ((1, 2, 3), 4)]
[((0, 1, 2, 3), 5), ((0, 1, 2), 4), ((0, 1, 3), 4), ((0, 2, 3), 4), ((1, 2, 3), 4)]
[]


In [6]:
# Allocating Study Groups
def alloc_study_groups(zbinis):
    # create a list for storing valid groups
    groups = []
    # get the possible groups using the possible_study_groups function
    possible_groups = possible_study_groups(zbinis)
    # set origin union set from possible_groups named unions
    unions = set(possible_groups[0][0])
    # the first group is always valid
    groups.append(possible_groups[0][0])
    # iterate through the rest of the possible groups
    for i, j in possible_groups[1:]:
        # check if the intersection of the group and the union is empty
        if len(set(unions).intersection(set(i))) == 0:
            groups.append(i)
            # update unions set
            unions = unions.union(set(i))
    return groups

print(alloc_study_groups([(198, ['FoC']), (198, ['FoC']), (138, ['FoC']), (14, ['FoC'])]))
# [(0, 2, 3)]
print(alloc_study_groups([(198, ['FoC']), (138, ['FoC', 'Calc 1']), (14, ['FoC', 'Calc 1']), (66, ['FoC', 'Calc 1'])]))
# [(0, 1, 2, 3)]
print(alloc_study_groups([(198, ['FoC', 'Logic']), (138, ['Calc 1']), (14, ['Calc 1']), (66, ['FoC', 'Logic']), (10, ['FoC', 'Logic']), (142, ['FoC', 'Logic']), (66, ['Calc 1'])]))
# [(0, 3, 4, 5), (1, 2, 6)]

[(0, 2, 3)]
[(0, 1, 2, 3)]
[(0, 3, 4, 5), (1, 2, 6)]
